## Requirements & Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install -q -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Config & Training 

In [5]:
from huggingface_hub import hf_hub_download

realistic_vision_path = hf_hub_download(repo_id="SG161222/Realistic_Vision_V5.1_noVAE", filename="Realistic_Vision_V5.1-inpainting.safetensors")
vae_path = hf_hub_download(repo_id="stabilityai/sd-vae-ft-mse-original", filename="vae-ft-mse-840000-ema-pruned.safetensors")

In [6]:
import ml_collections

config = ml_collections.ConfigDict()

## General
config.seed = 2808
config.logdir = "logs"
config.global_caption = "white-facades"
config.device = 'cuda'

## Dataset
config.dataset = dataset = ml_collections.ConfigDict()
# Roboflow API key
dataset.roboflow_api_key = 'HNXIsW3WwnidNDQZHexX'
# Roboflow workspace
dataset.roboflow_workspace = 'arked'
# Roboflow project name
dataset.project_name = 'white-facades'
# Roboflow dataset version
dataset.dataset_version = 1
# Local path where the dataset is downloaded
dataset.data_root = 'data'

# Prompt
config.prompt = prompt = ml_collections.ConfigDict()
# Global caption to add to every prompt
prompt.global_caption = 'white facades'

# Model
config.model = model = ml_collections.ConfigDict()
model.model_path = realistic_vision_path
model.vae_path = vae_path

## Training
config.train = train = ml_collections.ConfigDict()

# LoRA hyperparameters
train.lora_rank = 8
train.lora_scale = 1.0
train.lora_dropout_p = 0.1
train.train_unet = True
train.train_text_encoder = True
train.unet_lr = 2e-4
train.text_encoder_lr = 2e-4
train.mask_temperature = 1.0

# train batch size.
train.train_batch_size = 2
# eval batch size.
train.eval_batch_size = 1
# number of steps between saving model checkpoints.
train.save_freq = 100
# number of checkpoints to keep before overwriting old ones.
train.num_checkpoint_limit = 5
# mixed precision training. options are "fp16", "bf16", and "no". half-precision speeds up training significantly.
train.mixed_precision = "no"
# whether to use gradient checkpointing to reduce memory usage.
train.gradient_checkpointing = False
# number of gradient accumulation steps.
train.gradient_accumulation_steps = 1
# whether to use a learning rate schedule.
train.use_scheduler = True
# learning rate schedule type.
train.scheduler_type = "cosine_with_restarts"
# number of warmup steps for the learning rate schedule.
train.scheduler_warmup_steps = 500
# number of steps to train for.
train.total_steps = 4000
# learning rate.
train.learning_rate = 1e-4
# Adam beta1.
train.adam_beta1 = 0.9
# Adam beta2.
train.adam_beta2 = 0.999
# Adam weight decay.
train.adam_weight_decay = 1e-4
# Adam epsilon.
train.adam_epsilon = 1e-8
# clip gradients.
train.clip_gradients = True
# clip gradients max norm.
train.clip_gradients_max_norm = 1.0
# use xformers efficient attention
train.use_xformers = True
# checkpoint folder
train.checkpoint_folder = "checkpoints"

## Evaluation
config.eval = eval = ml_collections.ConfigDict()
# number of validation sample steps.
eval.num_eval_steps = 20
# whether to use a validation set.
eval.use_validation = True
# the prompts to use for validation.
eval.prompts = ['']
# the strengths to use for validation.
eval.strenghts = [1.0]

## Wandb
# whether to use wandb.
config.log_wandb = True
# wandb project name.
config.wandb = wandb = ml_collections.ConfigDict()
# wandb project name.
wandb.project_name = "lora"
# wandb entity name.
wandb.entity_name = "arked"
# wandb run name. (if not set is assigned automatically)
wandb.run_name = None
# wandb tags.
wandb.tags = ["lora"]

In [9]:
from src.training import train

train(config)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to data in yolov7pytorch:: 100%|██████████| 775/775 [00:01<00:00, 733.10it/s]


loading VAE...
loading model...


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: maidacundo (arked). Use `wandb login --relogin` to force relogin


Unet LoRA params: 2492928
CLIP LoRA params: 589824
Directory 'checkpoints' created.


Steps:   0%|          | 0/4000 [00:00<?, ?it/s]